In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

path="/content/drive/MyDrive/Human RNA/posneg.csv"
df= pd.read_csv(path)
path1="/content/drive/MyDrive/Human RNA/Ind-posneg.csv"
df1= pd.read_csv(path1)

df.describe()

,Sequences,Labels
count,1482,1482
unique,1482,2
top,AUGCAUUAGCCUUGUGGCUAGAACACCCUCUUCCUACCUCU,Positive
freq,1,741


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
from keras.preprocessing import text, sequence
import re

In [ ]:
print(df[ df['Labels'] == 'Positive'].size)
print(df[ df['Labels'] == 'Negative'].size)

print(df1[ df1['Labels'] == 'Positive'].size)
print(df1[ df1['Labels'] == 'Negative'].size)
for idx,row in df.iterrows():
    row[0] = str(row[0]).replace('rt','')
df.head()

1482
1482
668
6680


,Sequences,Labels
0,AUGCAUUAGCCUUGUGGCUAGAACACCCUCUUCCUACCUCU,Positive
1,UUCUUUUUUUUGUUUCAGAAGAACUGGACGGGGCUGGAGGA,Positive
2,AGGAACCCCCUGAACCCCAAGAGAGGGAGGACCAGGAUCCG,Positive
3,UUUUAGUUAAACGUUGAGGAGAAAAAAAAAAAAGGCUUUUC,Positive
4,GGCCCCAGUGCAGUACCCCAGCUCAUGGGGGACUCAGCAAG,Positive


In [ ]:
max_fatures = 500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Sequences'].values)
X = tokenizer.texts_to_sequences(df['Sequences'].values)
X = pad_sequences(X)

In [ ]:
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df1['Sequences'].values)
A = tokenizer.texts_to_sequences(df1['Sequences'].values)
A = pad_sequences(A)

In [ ]:
Y = pd.get_dummies(df['Labels']).values
B = pd.get_dummies(df1['Labels']).values
X_train = X
Y_train = Y
X_test = A
Y_test = B

In [ ]:
from tensorflow.keras.optimizers import SGD
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 128)            64000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 1, 128)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 319,194
Trainable params: 319,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 32
history=model.fit(X_train, Y_train, epochs =100, batch_size=batch_size, verbose = 2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Epoch 1/100
47/47 - 3s - loss: 0.6155 - accuracy: 0.7834 - 3s/epoch - 58ms/step
Epoch 2/100
47/47 - 0s - loss: 0.5055 - accuracy: 0.8367 - 254ms/epoch - 5ms/step
Epoch 3/100
47/47 - 0s - loss: 0.4371 - accuracy: 0.8367 - 268ms/epoch - 6ms/step
Epoch 4/100
47/47 - 0s - loss: 0.4068 - accuracy: 0.8360 - 232ms/epoch - 5ms/step
Epoch 5/100
47/47 - 0s - loss: 0.3926 - accuracy: 0.8360 - 250ms/epoch - 5ms/step
Epoch 6/100
47/47 - 0s - loss: 0.3859 - accuracy: 0.8367 - 261ms/epoch - 6ms/step
Epoch 7/100
47/47 - 0s - loss: 0.3879 - accuracy: 0.8367 - 264ms/epoch - 6ms/step
Epoch 8/100
47/47 - 0s - loss: 0.3806 - accuracy: 0.8367 - 273ms/epoch - 6ms/step
Epoch 9/100
47/47 - 0s - loss: 0.3802 - accuracy: 0.8360 - 262ms/epoch - 6ms/step
Epoch 10/100
47/47 - 0s - loss: 0.3780 - accuracy: 0.8367 - 250ms/epoch - 5ms/step
Epoch 11/100
47/47 - 0s - loss: 0.3807 - accuracy: 0.8367 - 259ms/epoch - 6ms/step
Epoch 12/100
47/47 - 0s - loss: 0.3772 - accuracy: 0.8367 - 280ms/epoch - 6ms/step
Epoch 13/100
47

In [ ]:
model.save_weights("model.h5")

In [ ]:
import numpy as np
Y_pred = np.argmax(model.predict(X_test,batch_size = batch_size), axis=-1)

In [ ]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix \n",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix 
 [[3175  165]
 [   0  334]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      3340
           1       0.67      1.00      0.80       334

    accuracy                           0.96      3674
   macro avg       0.83      0.98      0.89      3674
weighted avg       0.97      0.96      0.96      3674



In [ ]:
from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(df_test.true, df_test.pred)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(df_test.true, df_test.pred)
from sklearn.metrics import confusion_matrix as cm
cm(df_test.true, Y_pred)
print(classification_report(df_test.true, Y_pred))
tp, fn, fp, tn = confusion_matrix(df_test.true, Y_pred).ravel()
print("MCC ---> {0}".format(mc))
print("Confusion Matrix. tn, fp, fn, tp ---> ", tp, fn, fp, tn)
print("Precision --->TP/TP+Fp ", tp/(tp+fp))
print("Recall - SN - Sensitivity --->TP/TP+FN ", tp/(tp+fn))
print("Specificity - SP ---> ", tn/(tn+fp))
print("Balanced Accuracy ---> ", ((tp/(tp+fn))+(tn/(tn+fp)))/2)
print("Jaccard Index --->TP/TP+FN+FP ", tp/(tp+fn+fp))

MCC:  0.7976669386969604
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      3340
           1       0.67      1.00      0.80       334

    accuracy                           0.96      3674
   macro avg       0.83      0.98      0.89      3674
weighted avg       0.97      0.96      0.96      3674

MCC ---> 0.7976669386969604
Confusion Matrix. tn, fp, fn, tp --->  3175 165 0 334
Precision --->TP/TP+Fp  1.0
Recall - SN - Sensitivity --->TP/TP+FN  0.9505988023952096
Specificity - SP --->  1.0
Balanced Accuracy --->  0.9752994011976048
Jaccard Index --->TP/TP+FN+FP  0.9505988023952096
